In [21]:
import numpy as np
import torch as th
import time, json, sys, os, tqdm
import argparse
import torchvision
from PIL import Image
sys.path.append("/home/mint/Dev/DiFaReli/difareli-faster/sample_scripts/sample_utils/")
import mani_utils, params_utils

sample_json = '../extra_shadow_samples.json'
set_ = 'train'
with open(sample_json) as f:
    pairs = json.load(f)['pair']
    pairs = [{'src': p['src'], 'dst': p['dst']} for _, p in pairs.items()]

import misc
deca_params, avg_dict = misc.load_deca_params(deca_dir=f'/data/mint/DPM_Dataset/ffhq_256_with_anno/params/{set_}', cfg=None)


Loading deca params...: 100%|██████████| 10/10 [00:40<00:00,  4.10s/it]


In [48]:
n_frames = 20
rotate_sh_axis = 2

def proc_params(params, pair):
    pair_name = f"{pair['src'].split('.')[0]}_{pair['dst'].split('.')[0]}"
    out = {}
    for k in params.keys():
        if k == 'light':
            cond = {}
            cond[k] = th.tensor(params[k][None, ...])
            params[k] = mani_utils.rotate_sh(cond, src_idx=0, n_step=n_frames, axis=rotate_sh_axis)['light']
        else:
            params[k] = np.repeat(params[k][None, ...], repeats=n_frames, axis=0)
        out[k] = params[k]
    return out

proc_out = {}
fo = {}
save_params_folder = f"/data/mint/DPM_Dataset/Generated_Dataset_TargetLight/Generated_Dataset_cast_shadows_extra_rotate/params_recreate/{set_}"
os.makedirs(save_params_folder, exist_ok=True)
fo_shape = open(f"{save_params_folder}/ffhq-{set_}-shape-anno.txt", "w")
fo_exp = open(f"{save_params_folder}/ffhq-{set_}-exp-anno.txt", "w")
fo_pose = open(f"{save_params_folder}/ffhq-{set_}-pose-anno.txt", "w")
fo_light = open(f"{save_params_folder}/ffhq-{set_}-light-anno.txt", "w")
fo_cam = open(f"{save_params_folder}/ffhq-{set_}-cam-anno.txt", "w")
fo_detail = open(f"{save_params_folder}/ffhq-{set_}-detail-anno.txt", "w")
fo_tform = open(f"{save_params_folder}/ffhq-{set_}-tform-anno.txt", "w")
fo_albedo = open(f"{save_params_folder}/ffhq-{set_}-albedo-anno.txt", "w")
fo_faceemb = open(f"{save_params_folder}/ffhq-{set_}-faceemb-anno.txt", "w")
fo_shadow = open(f"{save_params_folder}/ffhq-{set_}-shadow-anno.txt", "w")

fo_dict = {'shape':fo_shape, 'exp':fo_exp, 'pose':fo_pose, 
        'light':fo_light, 'cam':fo_cam, 'detail':fo_detail,
        'tform':fo_tform, 'albedo':fo_albedo, 'shadow':fo_shadow, 'faceemb':fo_faceemb}

# fo_dict[fok].write(" ".join([str(x) for x in params]) + "\n")
for p in tqdm.tqdm(pairs):
    tmp_dict = proc_params(deca_params[p['src']].copy(), pair=p)
    for k in tmp_dict.keys():
        # print(tmp_dict[k].shape, tmp_dict['light'].shape, tmp_dict['light'])
        for i in range(n_frames):
            if i == 0:
                name = f"{p['src'].split('.')[0]}"
                ext = ".png"
            else:
                name = f"{p['src'].split('.')[0]}_{p['dst'].split('.')[0]}_f{i}"
                ext = ".png"
            fo_dict[k].write(name + f"{ext} ")
            fo_dict[k].write(" ".join([str(x) for x in tmp_dict[k][i].flatten()]) + "\n")
            # fo_dict[fok].write(" ".join([str(x) for x in tmp_dict[k][fok].flatten()]) + "\n")

for k in fo_dict.keys():
    fo_dict[k].close()

  0%|          | 0/500 [00:00<?, ?it/s]/home/mint/Dev/DiFaReli/difareli-faster/sample_scripts/sample_utils/mani_utils.py:323: UserWarning: Gimbal lock detected. Setting third angle to zero since it is not possible to uniquely determine all angles.
  return rot.as_euler('zyz', degrees=True)
100%|██████████| 500/500 [00:09<00:00, 50.26it/s]
